### 1. num_cell과 num_level에 따른 적절한 max_level은 아래와 같다.
아래와 같은 표를 만들어서 한번 볼 필요가 있는 이유는, 내가 max_level(하나의 weight를 표현하는데 사용 할 총레벨 수)을 100이라고 잡았다고 치자, 그리고 셀하나에 8레벨을 써, 근데 2 cell을 쓰면 8^2=64레벨밖에 표현 못하기 때문에 3 cells을 써야하는데 그렇게 되면 8^3=512레벨을 표현하게 된다. 이런 potential level(잠재적으로 실현가능한 max_level)을 가진 녀석을 고작 100레벨 표현하는데 쓰는건 비효율적이고 손해다. 그래서 각 세팅에 따른 적절한 max_level을 알아두는 것이 좋다.
MNIST가 6비트=64레벨에서 94프로 학습을 이룰 수 있으니까 64레벨을 이루려면..이게 좋다! 이런 식으로 감을 잡을 수 있지않을까?


In [4]:
import numpy as np
# import pandas as pd
max_numoflevel_per_cell=20
max_numofcell_per_weight=10
table=np.zeros([max_numofcell_per_weight,max_numoflevel_per_cell],dtype=np.int32)
for num_level in range(1,max_numoflevel_per_cell+1):
    for num_cell in range(1,max_numofcell_per_weight+1):
        table[num_cell-1,num_level-1]=num_level**num_cell
print(table[0,:])
print(table[1,:])
print(table[2,:])
print(table[3,:])

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
[  1   4   9  16  25  36  49  64  81 100 121 144 169 196 225 256 289 324
 361 400]
[   1    8   27   64  125  216  343  512  729 1000 1331 1728 2197 2744
 3375 4096 4913 5832 6859 8000]
[     1     16     81    256    625   1296   2401   4096   6561  10000
  14641  20736  28561  38416  50625  65536  83521 104976 130321 160000]


### 2. 사소한 팁 1:브로드캐스팅
참묘하게 직관적이란 말이야, 근데 원리를 딱 한마디로 설명은 못하겠다.

In [6]:
a=np.zeros(shape=[3,4,5])
a[:,:,0]=0
a[:,:,1]=1
a[:,:,2]=2
a[:,:,3]=3
a[:,:,4]=4
print(a)
np.sum(a*np.array([0,1,2,3,4]),axis=-1)  #axis=-1도 통하는구나

[[[0. 1. 2. 3. 4.]
  [0. 1. 2. 3. 4.]
  [0. 1. 2. 3. 4.]
  [0. 1. 2. 3. 4.]]

 [[0. 1. 2. 3. 4.]
  [0. 1. 2. 3. 4.]
  [0. 1. 2. 3. 4.]
  [0. 1. 2. 3. 4.]]

 [[0. 1. 2. 3. 4.]
  [0. 1. 2. 3. 4.]
  [0. 1. 2. 3. 4.]
  [0. 1. 2. 3. 4.]]]


array([[30., 30., 30., 30.],
       [30., 30., 30., 30.],
       [30., 30., 30., 30.]])

### 사소한 팁 2: random function 
randomness의 generation결과는 텐서가 추가 됨에 따라 달라진다. 그래서 만약 flow에 다른 텐서를 추가했다면 그 틀이 실질적인 영향을 주지는 않더라도 randomness의 generation결과가 다르기 때문에 random seed를 고정해도 기존과 같은 결과를 얻을 수 없다.

In [13]:
import tensorflow as tf
import numpy as np
tf.reset_default_graph()
tf.set_random_seed(333)
np.random.seed(333)
a=tf.random_normal(shape=[3,3])
b=np.random.normal(size=[3,3])
with tf.Session() as sess:
    print("a=",sess.run(a))
    print("b=",b)


a= [[-0.6436417  -0.94902575  0.3808462 ]
 [-0.40808907 -0.90408695  0.7453628 ]
 [-0.44379586  1.0692946   0.02469475]]
b= [[ 1.71718429  0.32469333 -0.4745434 ]
 [-0.1379183   0.52493957 -0.35783583]
 [-0.72820492 -0.64208645 -0.31704766]]


In [40]:
tf.reset_default_graph()
tf.set_random_seed(333)
np.random.seed(333)
c=tf.Variable(tf.zeros(shape=[3,3]))
# d=tf.constant(value=tf.zeros(shape=[3,3]))
a=tf.random_normal(shape=[3,1,3])
b=np.random.normal(size=[3,1,3])
with tf.Session() as sess:
    print("a=",sess.run(a))
    print("b=",b)

a= [[[-1.1583331  -1.9289364   0.1853901 ]]

 [[-0.32068866  1.3227402   0.5897821 ]]

 [[-1.3542254  -2.1340234  -0.8468843 ]]]
b= [[[ 1.71718429  0.32469333 -0.4745434 ]]

 [[-0.1379183   0.52493957 -0.35783583]]

 [[-0.72820492 -0.64208645 -0.31704766]]]


### 3. split and convert 함수 버전1
버전1인데..ㅎ 실행은 안된다, 안되는 이유는 적절하지 못한 방법으로 numpy와 tensorflow를 혼용했기 때문이다. 이 부분이 텐서플로우를 어렵게 만든다.
텐서플로우의 텐서는 기본적으로 삭제되고 새로 생성되고를 반복하는게 아니다. 이 말이 무슨 말이냐면,<br> 
1) 일반 코드로 iteration을 한다고 하면 매번 특정 matrix를 새로 만들어 쓰고 버리고를 반복하지만 텐서플로우는 틀을 만들어놓고, 그 틀에 맞춰 flow를 주고받는다.<br>
2) numpy를 코드의 맨 앞부분에 써서 flow의 인풋으로 넣는건 가능하다. 그런데 아마도 flow중간에 numpy를 넣는건 안되는거 같다.<br>
이게 무슨 말인지는 내가 했지만 나도 잘모른다, 엄밀하지 못한데 어쨌든 이런 느낌이다. 그래서 그 flow를 정해줘야한다.

덧1:고치다보니 여기다 그냥 고쳤네..버전1, numpy랑 섞어쓴건 없어진 셈이다.

In [58]:
import numpy as np
import tensorflow as tf
def split_level(level,array_shape, num_cell=None,num_level=None,mode=0):
    assert num_cell!=None and num_level!=None,"Please input num_cell and num_level"
    splited_level = tf.Variable(tf.transpose(tf.zeros(shape=array_shape)),dtype=tf.float32)
    level = tf.transpose(tf.cast(level,dtype=tf.float32))
    if mode==0:
        quotient=level  #level-1 if level=1,2,3,4,5,6...
        for i in range(num_cell):
            print(level)
            print(i,(quotient % num_level))
            print(splited_level[num_cell - i - 1,:] )
            assign_splited=tf.assign(splited_level[num_cell - i - 1,:], tf.cast(quotient%num_level,dtype=tf.float32))
            with tf.control_dependencies([assign_splited]): 
                quotient = (quotient - splited_level[num_cell - i - 1, :]) / num_level
        print("Original mode")
    elif mode==1:
        print("Simplified mode")
        raise NotImplementedError
    else:
        raise ValueError("Error! Please select correct mode.")
    return tf.transpose(splited_level)

In [63]:
splited_level=split_level(5*np.ones([3,8]),[3,8,3],3,3)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(splited_level))

Tensor("transpose_31:0", shape=(8, 3), dtype=float32)
0 Tensor("mod_46:0", shape=(8, 3), dtype=float32)
Tensor("strided_slice_67:0", shape=(8, 3), dtype=float32)
Tensor("transpose_31:0", shape=(8, 3), dtype=float32)
1 Tensor("mod_48:0", shape=(8, 3), dtype=float32)
Tensor("strided_slice_70:0", shape=(8, 3), dtype=float32)
Tensor("transpose_31:0", shape=(8, 3), dtype=float32)
2 Tensor("mod_50:0", shape=(8, 3), dtype=float32)
Tensor("strided_slice_73:0", shape=(8, 3), dtype=float32)
Original mode
[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]


-왜 결과가 전부 0으로 나오지?
-내 코드에는 tf.cast가 왜 이리 많이 나오냐..

In [109]:
splited_level,probe_list,probe_list2=split_level(5*np.ones([3,8]),[3,8,3],3,3)
print(probe_list2)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
#     print(sess.run(remainder))
    print(splited_level)
    print(sess.run(splited_level)) # 원하지 않는 결과!
    print(sess.run(probe_list))
    print(sess.run(splited_level)) # 원하는 결과!

quotient= Tensor("truediv_111:0", shape=(8, 3), dtype=float32)
quotient= Tensor("truediv_112:0", shape=(8, 3), dtype=float32)
quotient= Tensor("truediv_113:0", shape=(8, 3), dtype=float32)
Original mode
[<tf.Variable 'Variable_60:0' shape=(3, 8, 3) dtype=float32_ref>, <tf.Variable 'Variable_60:0' shape=(3, 8, 3) dtype=float32_ref>, <tf.Variable 'Variable_60:0' shape=(3, 8, 3) dtype=float32_ref>]
Tensor("transpose_125:0", shape=(3, 8, 3), dtype=float32)
[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]
[array([[5., 5., 5.],
       [5., 5., 5.],
       [5., 5., 5.],
       [5., 5., 5.],
       [5., 5., 5.],
       [5., 5., 5.],
       [5., 5., 5.],
       [5., 5., 5.]], dtype=float32), array([[1., 1., 1.],
       [1., 1., 1.

In [147]:
def split_level(level,array_shape, num_cell=None,num_level=None,mode=0):
    assert num_cell!=None and num_level!=None,"Please input num_cell and num_level"
    splited_level = tf.Variable(tf.transpose(tf.zeros(shape=array_shape)),dtype=tf.float32,trainable=False)
    level = tf.transpose(tf.cast(level,dtype=tf.float32))
    if mode==0:
        quotient=level  #level-1 if level=1,2,3,4,5,6...
        for i in range(num_cell):
            assign_splited=tf.assign(splited_level[num_cell - i - 1,:], tf.cast(quotient%num_level,dtype=tf.float32))
            with tf.control_dependencies([assign_splited]): 
                quotient = (quotient - splited_level[num_cell - i - 1, :]) / num_level
        with tf.control_dependencies([quotient]):
            splited_level=tf.identity(splited_level)
        print("Original mode")
    elif mode==1:
        print("Simplified mode")
        raise NotImplementedError
    else:
        raise ValueError("Error! Please select correct mode.")
    return tf.transpose(splited_level)

def convert_level(splited_level, filter_shape, num_cell=None,num_level=None,mode=0):
    assert num_cell != None and num_level != None, "Please input num_cell and num_level"
    bit_weight=num_level**np.arange(num_cell-1,-1,-1)
    if mode==0:
        converted_level=tf.reduce_sum(splited_level*bit_weight,axis=-1)
    elif mode==1:
        raise NotImplementedError
    else:
        raise ValueError("Error! Please select correct mode.")
    return converted_level

In [151]:
randomint=np.random.randint(low=0,high=26,size=[5,4])
splited_level=split_level(randomint,[5,4,3],3,3)
converted_level=convert_level(splited_level,[5,4],num_cell=3,num_level=3)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print("randomint=",randomint.shape,"\n",randomint)
    print("splited_level=",splited_level.shape,"\n",sess.run(splited_level))
    print("converted_level=",converted_level.shape,"\n",sess.run(converted_level))

Original mode
randomint= (5, 4) 
 [[17 24  0 24]
 [ 5 22  9  9]
 [20 21 24 11]
 [ 6 18 21  6]
 [ 8  7 13 16]]
splited_level= (5, 4, 3) 
 [[[1. 2. 2.]
  [2. 2. 0.]
  [0. 0. 0.]
  [2. 2. 0.]]

 [[0. 1. 2.]
  [2. 1. 1.]
  [1. 0. 0.]
  [1. 0. 0.]]

 [[2. 0. 2.]
  [2. 1. 0.]
  [2. 2. 0.]
  [1. 0. 2.]]

 [[0. 2. 0.]
  [2. 0. 0.]
  [2. 1. 0.]
  [0. 2. 0.]]

 [[0. 2. 2.]
  [0. 2. 1.]
  [1. 1. 1.]
  [1. 2. 1.]]]
converted_level= (5, 4) 
 [[17. 24.  0. 24.]
 [ 5. 22.  9.  9.]
 [20. 21. 24. 11.]
 [ 6. 18. 21.  6.]
 [ 8.  7. 13. 16.]]


와 된다, 쩔었다..비교적 빨리 해결함, 역시 그냥 붙잡고 있는다고 되는게 아니야~<br>
꼭 기억하자, 텐서플로우는 절차적코드가 아니라, 코드를 통해 flow를 짜는거야, flow 못만들면 절차 다 소용없어!!